# Tannic_Sentiments

## A Sentiment Classifier that has been triained on wine reviews. 

In [24]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Softmax, Embedding, GlobalMaxPool1D, GlobalAvgPool1D, Concatenate, Bidirectional, SpatialDropout1D

In [8]:
wine_df = pd.read_csv('data/winemag-data-130k-v2.csv')
wine_df.sample(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
22992,22992,France,"Blueberry, blackberry and boysenberry all abou...",Père et Fils Syrah-Grenache,87,10.0,Languedoc-Roussillon,Pays d'Oc,NaN,Lauren Buzzeo,@laurbuzz,Laurent Miquel 2010 Père et Fils Syrah-Grenach...,Syrah-Grenache,Laurent Miquel
98998,98998,Portugal,"This just off-dry wine is fresh, young and fru...",NaN,86,8.0,Vinho Verde,NaN,NaN,Roger Voss,@vossroger,Aveleda 2014 White (Vinho Verde),Portuguese White,Aveleda
3322,3322,New Zealand,"A plump, nicely concentrated Sauvignon Blanc, ...",NaN,87,20.0,Canterbury,NaN,NaN,Joe Czerwinski,@JoeCz,Mt. Beautiful 2009 Sauvignon Blanc (Canterbury),Sauvignon Blanc,Mt. Beautiful


In [10]:
trainable_df = wine_df[['description', 'points']]
trainable_df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [14]:
tokenizer = Tokenizer()

In [15]:
tokenizer.fit_on_texts(trainable_df)

In [22]:
x = pad_sequences(tokenizer.texts_to_sequences(trainable_df), 50)
y = trainable_df.points

In [25]:
input = Input((50,), name='input')
embed = Embedding(len(tokenizer.word_index), 100)(index)